## Made for Practice and to find out some interesting facts. Feedback Appreciated.


**AGENDA FOR NOTEBOOK:**

1.Basic groupby observations and pandas data viz

2.Seaborn plots - distributions and categories

3.pairplots joint plots and more in-depth insights

4.Matrix plots and improving inferences

5.Plotly for interctive plotting

In [ ]:
import numpy as np
import pandas as pd
import os
import seaborn as sns
import matplotlib as plt
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
df = pd.read_csv('/kaggle/input/us-natural-disaster-declarations/us_disaster_declarations.csv')
df.head(10)

In [ ]:
df.info()

In [ ]:
sns.pairplot(df[['disaster_number','fy_declared','incident_type','declaration_request_number']]);

# How frequently do disasters occur statewide

In [ ]:
disaster_types = df['incident_type'].unique()
print("Types of Disasters reported:\n\n", disaster_types)
print("Occurrences:\n\n",df['incident_type'].value_counts())

plotting natural disasters over the years by count and frequency


In [ ]:
plt.pyplot.figure(figsize=(10,6))
sns.distplot(df['incident_type'].value_counts());

In [ ]:
m=df[['fy_declared','incident_type']].groupby('fy_declared').describe()['incident_type'].reset_index()
plt.pyplot.figure(figsize=(9,6))
sns.barplot(x='top',y='freq',data =m);

In [ ]:
m=df['fy_declared'].value_counts().reset_index()
m.head(10)

In [ ]:
sns.lmplot(x='index',y='fy_declared',data=m.sort_values(by='fy_declared'),
           aspect=1.7, height=5,markers=['o'], scatter_kws={'s':20});

## To check which states are prone to what kinds of Natural Disasters

In [ ]:
m=df[['state','incident_type']].groupby('state').describe()
m['incident_type'].sort_values(by='count',ascending=False).head(10)

Plotting count, unique and frequency from describe() function

In [ ]:
m=df[['state','incident_type']].groupby('state').describe()
sns.pairplot(m['incident_type'].reset_index(drop=True));

In [ ]:
m['incident_type'].sort_values(by='freq',ascending=False)['freq'].head(10)

States most frequently hit by one kind of natural disaster

# States at Maximum Risk

In [ ]:
p = df[['state','incident_type']].groupby('state').count()
p.reset_index(inplace=True)
p=p.sort_values(by='incident_type',ascending=False).head(10)
p

Plotting All cases reported of natural disasters per state

In [ ]:
plt.pyplot.figure(figsize=(16,8))
sns.barplot(x="state", y="incident_type", data=p,);

Texas TX seems to be the most disaster Prone state, Lets find out more:

In [ ]:
print("Texas mostly faces:\n")
df[['state','incident_type']].groupby('state').max().loc['TX']

Texas reports over time:

In [ ]:
m=df.query('state=="TX"')['fy_declared'].value_counts().reset_index()
m.head(10)

2005 and 2020 were the worst years, most of the natural disasters reported were in these times. Interestingly the rate of natural disasters in TX has increased almost proportionally with time. Plotted Below:

In [ ]:
plt.pyplot.figure(figsize=(24,12))
sns.barplot(x='index',y='fy_declared',data=m.sort_values(by='fy_declared'));

# What is the reason of such a huge number of reports in 2020

In [ ]:
df.query('state == "TX" & fy_declared=="2020"')['incident_type'].unique()

## What are the most disaster prone designated areas?

In [ ]:
print("Total Areas\n",df['designated_area'].nunique())

In [ ]:
df[['designated_area','incident_type']].groupby('designated_area').count().sort_values(by='incident_type',ascending=False).head(10)

From this we can see that TX as a state has encountered maximum natural disasters but the designated area -- Washington County that has encountered maximum natural disasters is not in TX. 

# Looking at the dates on which maximum Statewide Disasters were reported: 

In [ ]:
d = pd.to_datetime(df['declaration_date']).dt
df['year'] = d.year
df['month'] = d.month
df['day'] = d.day
df['time'] = d.time
del df['declaration_date']

In [ ]:
df['declaration_type'].value_counts()

In [ ]:
df[['year','month','declaration_type']].groupby(['year','month']).describe()

In [ ]:
m=df[['year','declaration_type']]
plt.pyplot.figure(figsize=(8,5))
sns.boxplot(x='declaration_type',y='year',data=m);

In [ ]:
plt.pyplot.figure(figsize=(8,5))
sns.barplot(x='index',y='declaration_type',data=df['declaration_type'].value_counts().reset_index());

# Some Date-wise observations

In [ ]:
m=df[['year','month','day','declaration_type']].groupby(['year','month','day']).count()
m.reset_index().sort_values(by='declaration_type',ascending=False)

2893 disasters were reported on 2020-03-13. Lets break up the date time features to make some new features. And let's see what happened on 13/03/20,13/01/1996,04/04/2020,11/9/2005

In [ ]:
#13/03/20
df.query('year=="{0}" & month=="{1}" & day =="{2}"'.format(2020,3,13))['declaration_title'].value_counts()

It seems like the maximum number of reports ever was on 13/03/2020 and the reason was obviously COVID-19 outbreak. Let's plot the number of corona reports in 2020 per state

In [ ]:
m = df.query('year=="{0}" & declaration_title=="Covid-19"'.format(2020))
plt.pyplot.figure(figsize=(20,10))
sns.countplot(x='state',data=m.sort_values(by='time'));

In [ ]:
#13/01/1996
df.query('year=="{0}" & month=="{1}" & day =="{2}"'.format(1996,1,13))['declaration_title'].unique()

In [ ]:
df.query('year=="{0}" & month=="{1}" & day =="{2}"'.format(2020,4,4))['declaration_title'].unique()

In [ ]:
#September 2005
m=df.query('year=="{0}" & month=="{1}"'.format(2005,9))
m['declaration_title'].value_counts()

In [ ]:
plt.pyplot.figure(figsize=(16,16))
sns.countplot(x='declaration_title',data=m.sort_values(by='time'),hue='state');

This is clear that all reported causes in 1996 were because of the blizzard of '96, and in 2020 major causes are flood and Covid 19
In september 2005 Howevr 3 hurricanes, and 2 tropical fires were the reason

# Let's get some insight into reports for which individual assistance or housing was not provided

In [ ]:
m = df.query('ia_program_declared == "0" & ih_program_declared == "0" & pa_program_declared =="0"')
m['incident_type'].value_counts()

These people were not provided individual assistance or any of the programs to be helped though they reported calamities, assuming they were ignored for reasons out of our analysis scope

In [ ]:
pt = df.pivot_table(values ='disaster_number', index = 'state', columns = 'year').fillna(0)
plt.pyplot.figure(figsize=(12,8))
sns.heatmap(pt, cmap='coolwarm');
plt.pyplot.savefig('heatmap.png')